In [4]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv('./.env')

True

In [ ]:
# 1. 데이터 로드 및 전처리
max_seq_length = 2048
dtype = None # Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정
load_in_4bit = True # 메모리 사용량을 줄이기 위해 4bit 양자화를 권장

df = pd.read_csv('merged_data.csv')
texts = df['context_learn'].to_pandas().tolist()
documents = [Document(page_content=text) for text in texts]

# 2. FastLanguageModel을 통한 임베딩 모델 설정
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# 3. Hugging Face 임베딩 설정
embedding = HuggingFaceEmbeddings(model_name="Qwen/Qwen2-7B-Instruct")

# 4. 임베딩 파일 로드 또는 생성
embedding_file = "embeddings.pkl"
if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        all_embeddings = pickle.load(f)
    print("임베딩이 파일에서 불러와졌습니다.")
else:
    # 임베딩 생성 및 저장
    all_embeddings = [embedding.embed_text(text) for text in texts]
    with open(embedding_file, "wb") as f:
        pickle.dump(all_embeddings, f)

# 5. FAISS 인덱스 생성 및 벡터 스토어 설정
embedding_dim = len(all_embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(all_embeddings).astype('float32'))
vectorstore = FAISS(embedding_function=embedding.embed_query, index=index, docstore={i: doc for i, doc in enumerate(documents)}, index_to_docstore_id={i: i for i in range(len(documents))})

# 6. LangChain Retriever 설정
retriever = vectorstore.as_retriever()

# 7. 프롬프트 템플릿 생성
prompt_template = """
당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.
당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요.
만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면
'주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다'라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)

# 8. LangChain HuggingFacePipeline을 사용하여 LLM 설정
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 9. RetrievalQA 체인 생성
qa_chain = RetrievalQA(
    llm=llm,
    retriever=retriever,
    prompt=prompt
)

# 10. 질문 설정 및 질의응답 수행
query = "김영삼이 언급된 회의 내용을 알려줘."
result = qa_chain.run({"question": query})
print("답변:", result)

# 11. 메모리 정리
torch.cuda.empty_cache()


In [22]:
### csv 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 로드
loader = CSVLoader(file_path="merged_data.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

In [11]:
len(embedding_model.embed_query("테스트"))

384

In [18]:
# 필요한 모듈 import
import os
import faiss
import numpy as np
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.docstore.in_memory import InMemoryDocstore

# CSV 파일 경로
csv_file_path = 'merged_data.csv'

# CSV 파일에서 context_learn 컬럼 로드
df = pd.read_csv(csv_file_path)
context_learn_texts = df['context_learn'].astype(str).tolist()
documents = [Document(page_content=text) for text in context_learn_texts]  # Document 객체 생성

# 디바이스 설정 (GPU 사용)
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 확인

# 임베딩 모델 설정 (다국어 모델)
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# 벡터 저장소 파일 경로
vectorstore_file = 'merged_data_faiss_index.index'

# FAISS 인덱스 설정
embed_dimension = len(embedding_model.embed_query("테스트"))  # 임베딩 차원 수 자동 설정
index = faiss.IndexFlatL2(embed_dimension)

# VectorStore 설정
if os.path.exists(vectorstore_file):
    index = faiss.read_index(vectorstore_file)
else:
    embeddings = embedding_model.embed_documents(context_learn_texts)
    embeddings = np.array(embeddings).astype('float32')  # numpy 배열로 변환
    index.add(embeddings)  # FAISS에 임베딩 추가
    faiss.write_index(index, vectorstore_file)  # 인덱스 파일 저장

# `docstore`와 `index_to_docstore_id` 설정
index_to_docstore_id = {i: i for i in range(len(documents))}
docstore = InMemoryDocstore({i: doc for i, doc in enumerate(documents)})  # InMemoryDocstore로 설정

vectorstore = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query  # 임베딩 함수 설정
)

# Retriever 설정
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 생성
prompt_template = """
당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.
주어진 문맥(context)을 사용하여 질문(question)에 답하세요.
문맥에서 답을 찾을 수 없거나 모르는 경우, '주어진 정보에서 답변을 찾을 수 없습니다'라고 답하세요.

# Question:
{question}

# Context:
{context}

# Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)

# LLM 모델 설정
llm_model_name = "Qwen/Qwen-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained(llm_model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map='auto', trust_remote_code=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,  # 생성할 최대 토큰 수
    temperature=0,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=pipe)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.53it/s]


In [20]:
# 질의응답 테스트
# 질문 설정 및 벡터화
query = "김영삼이 언급된 회의에 대해 알려줘."
query_embedding = embedding_model.embed_query(query)  # 질문을 벡터화

# FAISS 인덱스를 통해 유사 문서 검색
k = 5  # 검색할 상위 k개의 유사 문서 수
distances, indices = index.search(np.array([query_embedding], dtype='float32'), k)

# 검색된 유사 문서 출력
similar_docs = [documents[i] for i in indices[0]]
for i, doc in enumerate(similar_docs):
    print(f"유사 문서 {i+1}: {doc.page_content}\n")

result = qa_chain({"query": query})
print("답변:", result['result'])
print("출처 문서:", result['source_documents'])


유사 문서 1: 그 문의와 관련하여 추출된 내용은 다음과 같습니다.'또 따로 논의해야 되지 않나요? 그럼요. 지금 여러 가지 이슈가 있어요. 그 전에 김영식·김철민·이탄희, 저금리 전환대출 확대와 관련된 부분은 교육부에서 이탄희 의원안으로 정리해서 얘기하자라고 했는데 그 부분에 대한 논의는 없었어요. 그것은 수용한 건가요? 그렇지요. 그것을 수용했다 그러면 다음 이슈로, 지금 김영식·김철민·이탄희 안은 이탄희 안으로 정리하는 것으로 협의가 되면 김철민 의원안에서는 교육·훈련시설 이것은 빼고 그다음에 교육부가 제시한 대로 교육부장관이 정하는 교육훈련기관이라는 것을 추가로 넣어 주는 것으로 자구를 정리하면 될 거고요.' 이 부분은 김영식, 김철민, 이탄희 의원의 저금리 전환대출 확대와 관련된 안건이 교육부에서 이탄희 의원의 안으로 정리되어 협의에서 수용되었음을 보여줍니다.

유사 문서 2: 박주선 부의장님과 국회의 동료 의원 여러분, 이낙연 국무총리님, 그리고 국민 여러분께 전하는 바입니다. 제가 속한 민주평화당의 첫 대정부 질문에서 말씀드리고자 하는 중요한 사항은, 평창 올림픽을 계기로 한 남북 관계 개선의 일환으로 북한 방문단이 곧 방문할 예정이라는 소식입니다. 여기에는 북한 김정은 위원장의 동생 김여정이 포함되어 있다는 보도가 있습니다. 이는 남북 관계에서 매우 높은 수준의 참여이며, 이러한 북한의 고위급 인사 방문이 남북관계의 개선과 평화 분위기 조성에 큰 도움이 될 것이라는 기대감을 갖고 있습니다. 그러나 현재로서 국무총리님께서는 이러한 사실을 공식적으로 확인해주실 수는 없으며, 공식 발표가 있을 때까지는 구체적인 내용을 확인하기 어려운 상태임을 이해하고 있습니다. 앞으로 진행될 공식 발표를 통해, 김여정 씨의 방문이 확정될 경우, 이는 국내외적으로 중대한 의미를 가지며, 한반도와 국제사회에 어떤 영향을 미칠지 주목되는 상황입니다.

유사 문서 3: 경기 광명을의 양기대 위원께서 남부청장인 김원준 청장님과 김남현 북부청장님에게 감사와 격려의 말씀을 전한 

/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


답변: 
당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.
주어진 문맥(context)을 사용하여 질문(question)에 답하세요.
문맥에서 답을 찾을 수 없거나 모르는 경우, '주어진 정보에서 답변을 찾을 수 없습니다'라고 답하세요.

# Question:
김영삼이 언급된 회의에 대해 알려줘.

# Context:
그 문의와 관련하여 추출된 내용은 다음과 같습니다.'또 따로 논의해야 되지 않나요? 그럼요. 지금 여러 가지 이슈가 있어요. 그 전에 김영식·김철민·이탄희, 저금리 전환대출 확대와 관련된 부분은 교육부에서 이탄희 의원안으로 정리해서 얘기하자라고 했는데 그 부분에 대한 논의는 없었어요. 그것은 수용한 건가요? 그렇지요. 그것을 수용했다 그러면 다음 이슈로, 지금 김영식·김철민·이탄희 안은 이탄희 안으로 정리하는 것으로 협의가 되면 김철민 의원안에서는 교육·훈련시설 이것은 빼고 그다음에 교육부가 제시한 대로 교육부장관이 정하는 교육훈련기관이라는 것을 추가로 넣어 주는 것으로 자구를 정리하면 될 거고요.' 이 부분은 김영식, 김철민, 이탄희 의원의 저금리 전환대출 확대와 관련된 안건이 교육부에서 이탄희 의원의 안으로 정리되어 협의에서 수용되었음을 보여줍니다.

박주선 부의장님과 국회의 동료 의원 여러분, 이낙연 국무총리님, 그리고 국민 여러분께 전하는 바입니다. 제가 속한 민주평화당의 첫 대정부 질문에서 말씀드리고자 하는 중요한 사항은, 평창 올림픽을 계기로 한 남북 관계 개선의 일환으로 북한 방문단이 곧 방문할 예정이라는 소식입니다. 여기에는 북한 김정은 위원장의 동생 김여정이 포함되어 있다는 보도가 있습니다. 이는 남북 관계에서 매우 높은 수준의 참여이며, 이러한 북한의 고위급 인사 방문이 남북관계의 개선과 평화 분위기 조성에 큰 도움이 될 것이라는 기대감을 갖고 있습니다. 그러나 현재로서 국무총리님께서는 이러한 사실을 공식적으로 확인해주실 수는 없으며, 공식 발표가 있을 때까지는 구체적인 내용을 확인하

In [23]:
import pandas as pd
from langchain.schema import Document

# CSV 파일 로드
df = pd.read_csv("merged_data.csv")

# Document 객체 생성 및 각 열을 메타데이터로 추가
documents = []
for _, row in df.iterrows():
    metadata = row.to_dict()  # 각 열을 메타데이터로 변환
    documents.append(Document(page_content=metadata['context_learn'], metadata=metadata))


In [ ]:
documents